# Ask Reddit

### NLP Classification problem for troll detection 

In [52]:
import numpy as np 
import pandas as pd
import os
import math
import matplotlib.pyplot as plt
import nltk
import re

In [53]:
#loading train and test data into two different dataframes
data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
df = pd.DataFrame(data)
test_df = pd.DataFrame(test_data)
df.head()

,qid,question_text,target
0,a3dee568776c08512c89,What is the role of Lua in Civ4?,0
1,bdb84f519e7b46e7b7bb,What are important chapters in Kannada for 10 ...,0
2,29c88db470e2eb5c97ad,Do musicians get royalties from YouTube?,0
3,3387d99bf2c3227ae8f1,What is the difference between Scaling Social ...,0
4,e79fa5038f765d0f2e7e,Why do elevators go super slow right before th...,0


In [4]:
test_df.head()

,qid,question_text
0,0a824224322f0a36025f,Why is my fish tank so cloudy?
1,28af14c4e4777ce1273e,Are AAP supporters/leaders hypocrites?
2,6892a52c51103dd95044,Can you still get a ticket if you shut off you...
3,badd9e8886d73fc1fe4e,Why should any liberal or caring person want t...
4,4ef178f82a465e4804ae,How can I know who got into my PC using anydesk?


In [6]:
#Checking for missing data  
df.isna().sum()

qid              0
question_text    0
target           0
dtype: int64

In [7]:
test_df.isna().sum()

qid              0
question_text    0
dtype: int64

In [11]:
df_target = df['target']
df_qid = df['qid']
df.pop('target')#not needed in training df_qid = df['qid']

0         0
1         0
2         0
3         0
4         0
         ..
653056    0
653057    0
653058    0
653059    0
653060    0
Name: target, Length: 653061, dtype: int64

In [12]:
#Sperating the question_text column for preprocessing
df_text = df['question_text']
print(df_text)
test_df_text = test_df['question_text']
print(test_df_text)

0                          What is the role of Lua in Civ4?
1         What are important chapters in Kannada for 10 ...
2                  Do musicians get royalties from YouTube?
3         What is the difference between Scaling Social ...
4         Why do elevators go super slow right before th...
                                ...                        
653056    Which coaching centre is best in Delhi for IAS...
653057    Is their any API to check CIBIL score and down...
653058    In paranormal shows, why do spirits talk to me...
653059    How does Josh Gordon have such well developed ...
653060           In how many ways methane can be extracted?
Name: question_text, Length: 653061, dtype: object
0                            Why is my fish tank so cloudy?
1                    Are AAP supporters/leaders hypocrites?
2         Can you still get a ticket if you shut off you...
3         Why should any liberal or caring person want t...
4          How can I know who got into my PC usin

### Prepocessing

#### 1) Removing puncuation

In [23]:
import string
string.punctuation
def remove_punch(text):
    text_non = "".join([char for char in text if char not in string.punctuation])
    return text_non
df_text['removing'] = df['question_text'].apply(lambda x: remove_punch(x)) 
print(df_text['removing'])

0                           What is the role of Lua in Civ4
1         What are important chapters in Kannada for 10 ...
2                   Do musicians get royalties from YouTube
3         What is the difference between Scaling Social ...
4         Why do elevators go super slow right before th...
                                ...                        
653056    Which coaching centre is best in Delhi for IAS...
653057    Is their any API to check CIBIL score and down...
653058    In paranormal shows why do spirits talk to med...
653059    How does Josh Gordon have such well developed abs
653060            In how many ways methane can be extracted
Name: question_text, Length: 653061, dtype: object


In [24]:
test_df_text['removing'] = test_df['question_text'].apply(lambda x: remove_punch(x)) 
print(test_df_text['removing'])

0                             Why is my fish tank so cloudy
1                      Are AAP supportersleaders hypocrites
2         Can you still get a ticket if you shut off you...
3         Why should any liberal or caring person want t...
4           How can I know who got into my PC using anydesk
                                ...                        
653056    Can we directly apply for Facebook market spec...
653057                 What we do to remind physics concept
653058                         Is rooting Lenovo A269i safe
653059                   What is the longest river of India
653060    What should you do if somehow you find yoursel...
Name: question_text, Length: 653061, dtype: object


#### 2) Tokenizing

In [25]:
import re
def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens
df_text['remove'] = df_text['removing'].apply(lambda x: tokenize(x.lower()))
print(df_text['remove'])
test_df_text['remove'] = test_df_text['removing'].apply(lambda x: tokenize(x.lower()))
print(test_df_text['remove'])

0                  [what, is, the, role, of, lua, in, civ4]
1         [what, are, important, chapters, in, kannada, ...
2            [do, musicians, get, royalties, from, youtube]
3         [what, is, the, difference, between, scaling, ...
4         [why, do, elevators, go, super, slow, right, b...
                                ...                        
653056    [which, coaching, centre, is, best, in, delhi,...
653057    [is, their, any, api, to, check, cibil, score,...
653058    [in, paranormal, shows, why, do, spirits, talk...
653059    [how, does, josh, gordon, have, such, well, de...
653060    [in, how, many, ways, methane, can, be, extrac...
Name: question_text, Length: 653061, dtype: object
0                     [why, is, my, fish, tank, so, cloudy]
1                 [are, aap, supportersleaders, hypocrites]
2         [can, you, still, get, a, ticket, if, you, shu...
3         [why, should, any, liberal, or, caring, person...
4         [how, can, i, know, who, got, into, my,

#### 3) Removing Stopwords

In [26]:
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(list):
    text = [word for word in list if word not in stopword]
    return text
df_text['stop'] = df_text['remove'].apply(lambda x: remove_stopwords(x))
print(df_text['stop'])
test_df_text['stop'] = test_df_text['remove'].apply(lambda x: remove_stopwords(x))
print(test_df_text['stop'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/manaswitha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0                                         [role, lua, civ4]
1            [important, chapters, kannada, 10, icse, 2018]
2                      [musicians, get, royalties, youtube]
3         [difference, scaling, social, enterprises, soc...
4          [elevators, go, super, slow, right, doors, open]
                                ...                        
653056    [coaching, centre, best, delhi, ias, preparation]
653057    [api, check, cibil, score, download, credit, r...
653058    [paranormal, shows, spirits, talk, mediums, us...
653059                 [josh, gordon, well, developed, abs]
653060                     [many, ways, methane, extracted]
Name: question_text, Length: 653061, dtype: object
0                                      [fish, tank, cloudy]
1                      [aap, supportersleaders, hypocrites]
2         [still, get, ticket, shut, car, cop, flips, li...
3         [liberal, caring, person, want, stay, country,...
4                           [know, got, pc, using

#### 4) Stemming

#### 5) Lemmatizing

In [27]:
wn = nltk.WordNetLemmatizer()
def lemmatizing(list):
    text = [wn.lemmatize(word) for word in list]
    return text
df_text['finish'] = df_text['stop'].apply(lambda x: lemmatizing(x))
print(df_text['finish'])
test_df_text['finish'] = test_df_text['stop'].apply(lambda x: lemmatizing(x))
print(test_df_text['finish'])

0                                         [role, lua, civ4]
1             [important, chapter, kannada, 10, icse, 2018]
2                         [musician, get, royalty, youtube]
3         [difference, scaling, social, enterprise, soci...
4            [elevator, go, super, slow, right, door, open]
                                ...                        
653056     [coaching, centre, best, delhi, ia, preparation]
653057    [api, check, cibil, score, download, credit, r...
653058    [paranormal, show, spirit, talk, medium, using...
653059                  [josh, gordon, well, developed, ab]
653060                      [many, way, methane, extracted]
Name: question_text, Length: 653061, dtype: object
0                                      [fish, tank, cloudy]
1                       [aap, supportersleaders, hypocrite]
2         [still, get, ticket, shut, car, cop, flip, light]
3         [liberal, caring, person, want, stay, country,...
4                           [know, got, pc, using

In [28]:
#Joining the processed data back into a sentence for feature engineering
text1 = ([[' '.join(i)] for i in df_text['finish']])
test_text1 = ([[' '.join(i)] for i in test_df_text['finish']])

In [29]:
#Checking the joined data
print(text1[0])
print(test_text1[0])

['role lua civ4']
['fish tank cloudy']


In [30]:
#Adding the processed data into the original dataframe
df['d'] = pd.DataFrame(text1)
df.dropna(subset = ["d"], inplace=True)
print(df['d'])
type(df)
len(df['d'])

0                                             role lua civ4
1                    important chapter kannada 10 icse 2018
2                              musician get royalty youtube
3         difference scaling social enterprise social fr...
4                    elevator go super slow right door open
                                ...                        
653056            coaching centre best delhi ia preparation
653057    api check cibil score download credit report i...
653058    paranormal show spirit talk medium using image...
653059                        josh gordon well developed ab
653060                           many way methane extracted
Name: d, Length: 653061, dtype: object


653061

In [31]:
test_df['d'] = pd.DataFrame(test_text1)
test_df.dropna(subset = ["d"], inplace=True)
print(test_df['d'])
type(test_df)
len(test_df['d'])

0                                          fish tank cloudy
1                           aap supportersleaders hypocrite
2                  still get ticket shut car cop flip light
3         liberal caring person want stay country nine m...
4                                 know got pc using anydesk
                                ...                        
653056    directly apply facebook market specialist hyde...
653057                                remind physic concept
653058                            rooting lenovo a269i safe
653059                                  longest river india
653060                       somehow find outside hurricane
Name: d, Length: 653061, dtype: object


653061

In [32]:
df.head()

,qid,question_text,d
0,a3dee568776c08512c89,What is the role of Lua in Civ4?,role lua civ4
1,bdb84f519e7b46e7b7bb,What are important chapters in Kannada for 10 ...,important chapter kannada 10 icse 2018
2,29c88db470e2eb5c97ad,Do musicians get royalties from YouTube?,musician get royalty youtube
3,3387d99bf2c3227ae8f1,What is the difference between Scaling Social ...,difference scaling social enterprise social fr...
4,e79fa5038f765d0f2e7e,Why do elevators go super slow right before th...,elevator go super slow right door open


In [33]:
test_df.head()

,qid,question_text,d
0,0a824224322f0a36025f,Why is my fish tank so cloudy?,fish tank cloudy
1,28af14c4e4777ce1273e,Are AAP supporters/leaders hypocrites?,aap supportersleaders hypocrite
2,6892a52c51103dd95044,Can you still get a ticket if you shut off you...,still get ticket shut car cop flip light
3,badd9e8886d73fc1fe4e,Why should any liberal or caring person want t...,liberal caring person want stay country nine m...
4,4ef178f82a465e4804ae,How can I know who got into my PC using anydesk?,know got pc using anydesk


### Feature engineering

#### 1) Sentiment analysis

In [34]:
from textblob import TextBlob
df["sentiment"] = df['d'].apply(lambda x: TextBlob(x).sentiment.polarity)
df.head()

,qid,question_text,d,sentiment
0,a3dee568776c08512c89,What is the role of Lua in Civ4?,role lua civ4,0.000000
1,bdb84f519e7b46e7b7bb,What are important chapters in Kannada for 10 ...,important chapter kannada 10 icse 2018,0.400000
2,29c88db470e2eb5c97ad,Do musicians get royalties from YouTube?,musician get royalty youtube,0.000000
3,3387d99bf2c3227ae8f1,What is the difference between Scaling Social ...,difference scaling social enterprise social fr...,0.033333
4,e79fa5038f765d0f2e7e,Why do elevators go super slow right before th...,elevator go super slow right door open,0.079762


In [35]:
from textblob import TextBlob
test_df["sentiment"] = test_df['d'].apply(lambda x: TextBlob(x).sentiment.polarity)
test_df.head()

,qid,question_text,d,sentiment
0,0a824224322f0a36025f,Why is my fish tank so cloudy?,fish tank cloudy,0.0
1,28af14c4e4777ce1273e,Are AAP supporters/leaders hypocrites?,aap supportersleaders hypocrite,0.0
2,6892a52c51103dd95044,Can you still get a ticket if you shut off you...,still get ticket shut car cop flip light,0.4
3,badd9e8886d73fc1fe4e,Why should any liberal or caring person want t...,liberal caring person want stay country nine m...,0.0
4,4ef178f82a465e4804ae,How can I know who got into my PC using anydesk?,know got pc using anydesk,0.0


#### 2) Word count
#### 3) Character count
#### 4) Sentence count
#### 5) Average word length
#### 6) Average sentence length

In [36]:
df['word_count'] = df["d"].apply(lambda x: len(str(x).split(" ")))
df['char_count'] = df["d"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))
df['sentence_count'] = df["d"].apply(lambda x: len(str(x).split(".")))
df['avg_word_length'] = df['char_count'] / df['word_count']
df['avg_sentence_length'] = df['word_count'] / df['sentence_count']
df.head()

,qid,question_text,d,sentiment,word_count,char_count,sentence_count,avg_word_length,avg_sentence_length
0,a3dee568776c08512c89,What is the role of Lua in Civ4?,role lua civ4,0.000000,3,11,1,3.666667,3.0
1,bdb84f519e7b46e7b7bb,What are important chapters in Kannada for 10 ...,important chapter kannada 10 icse 2018,0.400000,6,33,1,5.500000,6.0
2,29c88db470e2eb5c97ad,Do musicians get royalties from YouTube?,musician get royalty youtube,0.000000,4,25,1,6.250000,4.0
3,3387d99bf2c3227ae8f1,What is the difference between Scaling Social ...,difference scaling social enterprise social fr...,0.033333,6,50,1,8.333333,6.0
4,e79fa5038f765d0f2e7e,Why do elevators go super slow right before th...,elevator go super slow right door open,0.079762,7,32,1,4.571429,7.0


In [37]:
test_df['word_count'] = test_df["d"].apply(lambda x: len(str(x).split(" ")))
test_df['char_count'] = test_df["d"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))
test_df['sentence_count'] = test_df["d"].apply(lambda x: len(str(x).split(".")))
test_df['avg_word_length'] = test_df['char_count'] / test_df['word_count']
test_df['avg_sentence_length'] = test_df['word_count'] / test_df['sentence_count']
test_df.head()

,qid,question_text,d,sentiment,word_count,char_count,sentence_count,avg_word_length,avg_sentence_length
0,0a824224322f0a36025f,Why is my fish tank so cloudy?,fish tank cloudy,0.0,3,14,1,4.666667,3.0
1,28af14c4e4777ce1273e,Are AAP supporters/leaders hypocrites?,aap supportersleaders hypocrite,0.0,3,29,1,9.666667,3.0
2,6892a52c51103dd95044,Can you still get a ticket if you shut off you...,still get ticket shut car cop flip light,0.4,8,33,1,4.125000,8.0
3,badd9e8886d73fc1fe4e,Why should any liberal or caring person want t...,liberal caring person want stay country nine m...,0.0,9,48,1,5.333333,9.0
4,4ef178f82a465e4804ae,How can I know who got into my PC using anydesk?,know got pc using anydesk,0.0,5,21,1,4.200000,5.0


In [38]:
df_text = df['question_text']
print(df_text)
test_df_text = test_df['question_text']
print(test_df_text)

0                          What is the role of Lua in Civ4?
1         What are important chapters in Kannada for 10 ...
2                  Do musicians get royalties from YouTube?
3         What is the difference between Scaling Social ...
4         Why do elevators go super slow right before th...
                                ...                        
653056    Which coaching centre is best in Delhi for IAS...
653057    Is their any API to check CIBIL score and down...
653058    In paranormal shows, why do spirits talk to me...
653059    How does Josh Gordon have such well developed ...
653060           In how many ways methane can be extracted?
Name: question_text, Length: 653061, dtype: object
0                            Why is my fish tank so cloudy?
1                    Are AAP supporters/leaders hypocrites?
2         Can you still get a ticket if you shut off you...
3         Why should any liberal or caring person want t...
4          How can I know who got into my PC usin

#### 7) Number of puncuation marks in a sentence

In [39]:
text = []
k = 0
length = []
punc = []
for i in df_text:
    text.append(i)
    res = len(i.split())
    length.append(float(str(res))/8.0)
    count = 0;
    for j in range (0, len (i)):   
    #Checks whether given character is a punctuation mark  
        if i[j] in ('!', "," ,"\'" ,";" ,"\"", ".", "-" ,"?"):  
            count = count + 1;  
    punc.append(float(count))

In [40]:
test_text = []
k = 0
test_length = []
test_punc = []
for i in test_df_text:
    text.append(i)
    res = len(i.split())
    test_length.append(float(str(res))/8.0)
    count = 0;
    for j in range (0, len (i)):   
    #Checks whether given character is a punctuation mark  
        if i[j] in ('!', "," ,"\'" ,";" ,"\"", ".", "-" ,"?"):  
            count = count + 1;  
    test_punc.append(float(count))

In [41]:
df.pop('qid')#not needed in training 
df.pop('d')#not needed in training 
d = df.pop('question_text')#not needed in training 
df['punc'] = punc
df['d'] = d
test_df.pop('qid')#not needed in training 
test_df.pop('d')#not needed in training 
test_d = test_df.pop('question_text')#not needed in training 
test_df['punc'] = test_punc
test_df['d'] = test_d

In [42]:
x = np.array(df)
print(x)

[[0.0 3 11 ... 3.0 1.0 'What is the role of Lua in Civ4?']
 [0.4 6 33 ... 6.0 1.0
  'What are important chapters in Kannada for 10 ICSE 2018?']
 [0.0 4 25 ... 4.0 1.0 'Do musicians get royalties from YouTube?']
 ...
 [0.0 9 51 ... 9.0 2.0
  'In paranormal shows, why do spirits talk to mediums using images instead of words?']
 [0.1 5 25 ... 5.0 1.0
  'How does Josh Gordon have such well developed abs?']
 [0.5 4 23 ... 4.0 1.0 'In how many ways methane can be extracted?']]


In [43]:
test_x = np.array(test_df)
print(test_x)

[[0.0 3 14 ... 3.0 1.0 'Why is my fish tank so cloudy?']
 [0.0 3 29 ... 3.0 1.0 'Are AAP supporters/leaders hypocrites?']
 [0.4 8 33 ... 8.0 1.0
  'Can you still get a ticket if you shut off your car before a cop flips his lights?']
 ...
 [0.5 4 22 ... 4.0 1.0 'Is rooting Lenovo A269i safe?']
 [0.0 3 17 ... 3.0 1.0 'What is the longest river of India?']
 [0.0 4 27 ... 4.0 1.0
  'What should you do if somehow you find yourself outside in a hurricane?']]


### Standardizing all the extra featues which we added 

In [49]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
x[:,[0]] = sc_X.fit_transform(x[:,[0]])
x[:,[1]] = sc_X.fit_transform(x[:,[1]])
x[:,[2]] = sc_X.fit_transform(x[:,[2]])
x[:,[3]] = sc_X.fit_transform(x[:,[3]])
x[:,[4]] = sc_X.fit_transform(x[:,[4]])
x[:,[5]] = sc_X.fit_transform(x[:,[5]])
x[:,[6]] = sc_X.fit_transform(x[:,[6]])

In [45]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
test_x[:,[0]] = sc_X.fit_transform(test_x[:,[0]])
test_x[:,[1]] = sc_X.fit_transform(test_x[:,[1]])
test_x[:,[2]] = sc_X.fit_transform(test_x[:,[2]])
test_x[:,[3]] = sc_X.fit_transform(test_x[:,[3]])
test_x[:,[4]] = sc_X.fit_transform(test_x[:,[4]])
test_x[:,[5]] = sc_X.fit_transform(test_x[:,[5]])
test_x[:,[6]] = sc_X.fit_transform(test_x[:,[6]])

In [46]:
print(x[:,[0,6]])

[[-0.3096983141354009 -0.5023223689806728]
 [1.046231271036715 -0.5023223689806728]
 [-0.3096983141354009 -0.5023223689806728]
 ...
 [-0.3096983141354009 0.3706774676451547]
 [0.029284082157628066 -0.5023223689806728]
 [1.3852136673297437 -0.5023223689806728]]


In [47]:
print(test_x[:,[0,6]])

[[-0.30991896155571236 -0.501798041786955]
 [-0.30991896155571236 -0.501798041786955]
 [1.0469327976044018 -0.501798041786955]
 ...
 [1.3861457373944304 -0.501798041786955]
 [-0.30991896155571236 -0.501798041786955]
 [-0.30991896155571236 -0.501798041786955]]


### Classification Model

In [48]:
#Test Train data
x_train = x
y_train =  df_target
x_test = test_x

In [50]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV

In [51]:
# Create Function Transformer to use Feature Union
def get_numeric_data(x):
    return [record[:-1].astype(float) for record in x]

def get_text_data(x):
    return [record[-1] for record in x]

transfomer_numeric = FunctionTransformer(get_numeric_data)
transformer_text = FunctionTransformer(get_text_data)

### Pipelining is used to analyze both numeric and text featues

#### We used a grid search to find the best logistic regression model 

In [55]:
# Create a pipeline to concatenate Tfidf Vector and Numeric data
# Use RandomForestClassifier as an example
pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector', transfomer_numeric)
            ])),
             ('text_features', Pipeline([
                ('selector', transformer_text),
                ('vec', CountVectorizer())
            ]))
         ])),
    ('clf', LogisticRegression())
])

In [58]:
# Grid Search Parameters for LogiaticRegression
#Creating a dictionary grid for grid search
#'clf__class_weight': [{0:x, 1:1.0-x} for x in weights]
param_grid = {'clf__class_weight':[{0:0.19, 1:0.81},{0:0.13, 1:0.87}],
             'clf__max_iter':[5000,7000],
             'clf__tol':[0.002,0.007],
             'clf__intercept_scaling':[11],
             'clf__solver':['liblinear'],
             'clf__random_state':[10] }



# Training config
#'Accuracy' : 'accuracy'
scoring = {'F1': 'f1_macro','Accuracy' : 'accuracy'}
refit = 'F1'

# Perform GridSearch
rf_model = GridSearchCV(pipeline, param_grid=param_grid, cv=5, scoring=scoring,refit=refit, n_jobs=-1, return_train_score=True, verbose=1)
rf_model.fit(x_train, y_train)
rf_best = rf_model.best_estimator_

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/Users/manaswitha/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [59]:
y_pred = rf_best.predict(x_test)
df_param = pd.DataFrame(rf_model.cv_results_) 
print(rf_model.best_estimator_)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('numeric_features',
                                                 Pipeline(steps=[('selector',
                                                                  FunctionTransformer(func=<function get_numeric_data at 0x7fbe1948c310>))])),
                                                ('text_features',
                                                 Pipeline(steps=[('selector',
                                                                  FunctionTransformer(func=<function get_text_data at 0x7fbddff45ee0>)),
                                                                 ('vec',
                                                                  CountVectorizer())]))])),
                ('clf',
                 LogisticRegression(class_weight={0: 0.19, 1: 0.81},
                                    intercept_scaling=11, max_iter=5000,
                                    random_state=10, solver='liblinea

#### Other models which we tried out

### Saving the predicted target variable

In [142]:
result = pd.read_csv("sample_submission.csv")
result_df = pd.DataFrame(result)
print(type(x_test))
#print(type(y_pred))
#print(x_reducedtest['keyValue'])
print(result_df)
result_df['target'] = y_pred
result_df['target'].value_counts()

<class 'numpy.ndarray'>
                         qid  target
0       0a824224322f0a36025f       0
1       28af14c4e4777ce1273e       0
2       6892a52c51103dd95044       0
3       badd9e8886d73fc1fe4e       0
4       4ef178f82a465e4804ae       0
...                      ...     ...
653056  c2a1f06f34b81ebf46b0       0
653057  1062e8840a6d64d3d638       0
653058  785bc805f28073e7f4a3       0
653059  a0c5dfc967b11c7b3d5b       0
653060  c8f7b7999cf971f549a5       0

[653061 rows x 2 columns]


0    601445
1     51616
Name: target, dtype: int64

In [143]:
result_df.to_csv('GridcvStandardscalar.csv',index = False)